In [1]:
import os
import cv2


In [ ]:
img = cv2.imread("data/images/wynonnaearp.jpg")
img.shape

In [2]:
path = "outputs/example_seg_img.jpg"
print(f"File exists: {os.path.exists(path)}")

File exists: True


In [3]:
img = cv2.imread(path)
if img is None:
    print("Image not found")
else:
    print("Image loaded")



Image loaded
